In [1]:
import pandas as pd
import numpy as np

In [2]:
gdp_data = pd.read_csv("../data/external/gdp/GDP_2018-2020.csv")

In [3]:
gdp_data

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,QGDP,TOT,PC_CHGPP,Q,2017-Q1,0.277019,NaN
1,AUS,QGDP,TOT,PC_CHGPP,Q,2017-Q2,0.684937,NaN
2,AUS,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.979421,NaN
3,AUS,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.485912,NaN
4,AUS,QGDP,TOT,PC_CHGPP,Q,2018-Q1,0.904628,NaN
...,...,...,...,...,...,...,...,...
130,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2019-Q3,0.254837,NaN
131,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2019-Q4,0.181867,NaN
132,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2020-Q1,-3.279383,NaN
133,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2020-Q2,-11.346776,NaN


In [4]:
aud_gdp = gdp_data[0:15]
aud_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,QGDP,TOT,PC_CHGPP,Q,2017-Q1,0.277019,NaN
1,AUS,QGDP,TOT,PC_CHGPP,Q,2017-Q2,0.684937,NaN
2,AUS,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.979421,NaN
3,AUS,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.485912,NaN
4,AUS,QGDP,TOT,PC_CHGPP,Q,2018-Q1,0.904628,NaN
5,AUS,QGDP,TOT,PC_CHGPP,Q,2018-Q2,0.872744,NaN
6,AUS,QGDP,TOT,PC_CHGPP,Q,2018-Q3,0.368405,NaN
7,AUS,QGDP,TOT,PC_CHGPP,Q,2018-Q4,0.186410,NaN
8,AUS,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.497128,NaN
9,AUS,QGDP,TOT,PC_CHGPP,Q,2019-Q2,0.649125,NaN


In [5]:
gbp_gdp = gdp_data[15:30]
gbp_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
15,GBR,QGDP,TOT,PC_CHGPP,Q,2017-Q1,0.496099,NaN
16,GBR,QGDP,TOT,PC_CHGPP,Q,2017-Q2,0.292921,NaN
17,GBR,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.423409,NaN
18,GBR,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.355002,NaN
19,GBR,QGDP,TOT,PC_CHGPP,Q,2018-Q1,0.069019,NaN
20,GBR,QGDP,TOT,PC_CHGPP,Q,2018-Q2,0.382632,NaN
21,GBR,QGDP,TOT,PC_CHGPP,Q,2018-Q3,0.596473,NaN
22,GBR,QGDP,TOT,PC_CHGPP,Q,2018-Q4,0.166752,NaN
23,GBR,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.552374,NaN
24,GBR,QGDP,TOT,PC_CHGPP,Q,2019-Q2,0.138397,NaN


In [6]:
jpn_gdp = gdp_data[30:45]
jpn_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
30,JPN,QGDP,TOT,PC_CHGPP,Q,2017-Q1,0.740644,NaN
31,JPN,QGDP,TOT,PC_CHGPP,Q,2017-Q2,0.260081,NaN
32,JPN,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.797642,NaN
33,JPN,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.402805,NaN
34,JPN,QGDP,TOT,PC_CHGPP,Q,2018-Q1,-0.034943,NaN
35,JPN,QGDP,TOT,PC_CHGPP,Q,2018-Q2,0.051794,NaN
36,JPN,QGDP,TOT,PC_CHGPP,Q,2018-Q3,-0.653422,NaN
37,JPN,QGDP,TOT,PC_CHGPP,Q,2018-Q4,0.453693,NaN
38,JPN,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.577410,NaN
39,JPN,QGDP,TOT,PC_CHGPP,Q,2019-Q2,0.077577,NaN


In [7]:
cad_gdp = gdp_data[60:75]
cad_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
60,CAN,QGDP,TOT,PC_CHGPP,Q,2017-Q1,1.086920,NaN
61,CAN,QGDP,TOT,PC_CHGPP,Q,2017-Q2,1.023044,NaN
62,CAN,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.421873,NaN
63,CAN,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.517991,NaN
64,CAN,QGDP,TOT,PC_CHGPP,Q,2018-Q1,0.657094,NaN
65,CAN,QGDP,TOT,PC_CHGPP,Q,2018-Q2,0.647084,NaN
66,CAN,QGDP,TOT,PC_CHGPP,Q,2018-Q3,0.684309,NaN
67,CAN,QGDP,TOT,PC_CHGPP,Q,2018-Q4,0.276437,NaN
68,CAN,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.088235,NaN
69,CAN,QGDP,TOT,PC_CHGPP,Q,2019-Q2,1.069136,NaN


In [8]:
usd_gdp = gdp_data[75:90]
usd_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
75,USA,QGDP,TOT,PC_CHGPP,Q,2017-Q1,0.565669,NaN
76,USA,QGDP,TOT,PC_CHGPP,Q,2017-Q2,0.426944,NaN
77,USA,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.728834,NaN
78,USA,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.955677,NaN
79,USA,QGDP,TOT,PC_CHGPP,Q,2018-Q1,0.931679,NaN
80,USA,QGDP,TOT,PC_CHGPP,Q,2018-Q2,0.668628,NaN
81,USA,QGDP,TOT,PC_CHGPP,Q,2018-Q3,0.525196,NaN
82,USA,QGDP,TOT,PC_CHGPP,Q,2018-Q4,0.328321,NaN
83,USA,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.725123,NaN
84,USA,QGDP,TOT,PC_CHGPP,Q,2019-Q2,0.370716,NaN


In [9]:
chf_gdp = gdp_data[90:105]
chf_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
90,CHE,QGDP,TOT,PC_CHGPP,Q,2017-Q1,0.182532,NaN
91,CHE,QGDP,TOT,PC_CHGPP,Q,2017-Q2,0.270721,NaN
92,CHE,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.653977,NaN
93,CHE,QGDP,TOT,PC_CHGPP,Q,2017-Q4,1.067956,NaN
94,CHE,QGDP,TOT,PC_CHGPP,Q,2018-Q1,1.116710,NaN
95,CHE,QGDP,TOT,PC_CHGPP,Q,2018-Q2,0.978363,NaN
96,CHE,QGDP,TOT,PC_CHGPP,Q,2018-Q3,-0.161780,NaN
97,CHE,QGDP,TOT,PC_CHGPP,Q,2018-Q4,0.174014,NaN
98,CHE,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.169915,NaN
99,CHE,QGDP,TOT,PC_CHGPP,Q,2019-Q2,0.370392,NaN


In [10]:
nzd_gdp = gdp_data[105:120]
nzd_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
105,NZL,QGDP,TOT,PC_CHGPP,Q,2017-Q1,1.055088,NaN
106,NZL,QGDP,TOT,PC_CHGPP,Q,2017-Q2,1.096382,NaN
107,NZL,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.858552,NaN
108,NZL,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.869200,NaN
109,NZL,QGDP,TOT,PC_CHGPP,Q,2018-Q1,0.731806,NaN
110,NZL,QGDP,TOT,PC_CHGPP,Q,2018-Q2,1.128668,NaN
111,NZL,QGDP,TOT,PC_CHGPP,Q,2018-Q3,0.243979,NaN
112,NZL,QGDP,TOT,PC_CHGPP,Q,2018-Q4,1.112126,NaN
113,NZL,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.413104,NaN
114,NZL,QGDP,TOT,PC_CHGPP,Q,2019-Q2,0.393214,NaN


In [11]:
eur_gdp = gdp_data[120:135]
eur_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
120,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2017-Q1,0.777037,NaN
121,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2017-Q2,0.766840,NaN
122,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.780146,NaN
123,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.852729,NaN
124,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2018-Q1,0.282061,NaN
125,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2018-Q2,0.526933,NaN
126,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2018-Q3,0.157094,NaN
127,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2018-Q4,0.561289,NaN
128,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.561520,NaN
129,EU27_2020,QGDP,TOT,PC_CHGPP,Q,2019-Q2,0.259161,NaN


In [12]:
usd_gdp = usd_gdp.rename(columns={"TIME": "Time", "Value": "GDP"})
cad_gdp = cad_gdp.rename(columns={"TIME": "Time", "Value": "GDP"})
eur_gdp = eur_gdp.rename(columns={"TIME": "Time", "Value": "GDP"})
chf_gdp = chf_gdp.rename(columns={"TIME": "Time", "Value": "GDP"})
jpn_gdp = jpn_gdp.rename(columns={"TIME": "Time", "Value": "GDP"})
gbp_gdp = gbp_gdp.rename(columns={"TIME": "Time", "Value": "GDP"})
aud_gdp = aud_gdp.rename(columns={"TIME": "Time", "Value": "GDP"})
nzd_gdp = nzd_gdp.rename(columns={"TIME": "Time", "Value": "GDP"})
cad_gdp

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,Time,GDP,Flag Codes
60,CAN,QGDP,TOT,PC_CHGPP,Q,2017-Q1,1.086920,NaN
61,CAN,QGDP,TOT,PC_CHGPP,Q,2017-Q2,1.023044,NaN
62,CAN,QGDP,TOT,PC_CHGPP,Q,2017-Q3,0.421873,NaN
63,CAN,QGDP,TOT,PC_CHGPP,Q,2017-Q4,0.517991,NaN
64,CAN,QGDP,TOT,PC_CHGPP,Q,2018-Q1,0.657094,NaN
65,CAN,QGDP,TOT,PC_CHGPP,Q,2018-Q2,0.647084,NaN
66,CAN,QGDP,TOT,PC_CHGPP,Q,2018-Q3,0.684309,NaN
67,CAN,QGDP,TOT,PC_CHGPP,Q,2018-Q4,0.276437,NaN
68,CAN,QGDP,TOT,PC_CHGPP,Q,2019-Q1,0.088235,NaN
69,CAN,QGDP,TOT,PC_CHGPP,Q,2019-Q2,1.069136,NaN


In [13]:
usd_gdp = usd_gdp[{"Time", "GDP"}]
aud_gdp = aud_gdp[{"Time", "GDP"}]
cad_gdp = cad_gdp[{"Time", "GDP"}]
jpn_gdp = jpn_gdp[{"Time", "GDP"}]
nzd_gdp = nzd_gdp[{"Time", "GDP"}]
chf_gdp = chf_gdp[{"Time", "GDP"}]
eur_gdp = eur_gdp[{"Time", "GDP"}]
gbp_gdp = gbp_gdp[{"Time", "GDP"}]
gbp_gdp

,GDP,Time
15,0.496099,2017-Q1
16,0.292921,2017-Q2
17,0.423409,2017-Q3
18,0.355002,2017-Q4
19,0.069019,2018-Q1
20,0.382632,2018-Q2
21,0.596473,2018-Q3
22,0.166752,2018-Q4
23,0.552374,2019-Q1
24,0.138397,2019-Q2


In [14]:
def convert_quarter_to_date(currency):
    date_array = [""]*15
    index = 0
    for i in currency.iterrows():
        date = i[1]["Time"]
        if "Q1" in date:
            date = str(date[:5]) + "01-01"
        elif "Q2" in date:
            date = str(date[:5]) + "04-01"
        elif "Q3" in date:
            date = str(date[:5]) + "07-01"
        else:
            date = str(date[:5]) + "10-01"
        date_array[index] = date
        index += 1
    new_df = pd.DataFrame({"Time": date_array, "GDP": currency["GDP"]})
    return new_df

print(convert_quarter_to_date(gbp_gdp))

          Time        GDP
15  2017-01-01   0.496099
16  2017-04-01   0.292921
17  2017-07-01   0.423409
18  2017-10-01   0.355002
19  2018-01-01   0.069019
20  2018-04-01   0.382632
21  2018-07-01   0.596473
22  2018-10-01   0.166752
23  2019-01-01   0.552374
24  2019-04-01   0.138397
25  2019-07-01   0.497284
26  2019-10-01   0.017259
27  2020-01-01  -3.019094
28  2020-04-01 -18.752686
29  2020-07-01  16.003485


In [15]:
usd_gdp_date = convert_quarter_to_date(usd_gdp)
cad_gdp_date = convert_quarter_to_date(cad_gdp)
gbp_gdp_date = convert_quarter_to_date(gbp_gdp)
eur_gdp_date = convert_quarter_to_date(eur_gdp)
aud_gdp_date = convert_quarter_to_date(aud_gdp)
nzd_gdp_date = convert_quarter_to_date(nzd_gdp)
chf_gdp_date = convert_quarter_to_date(chf_gdp)
jpn_gdp_date = convert_quarter_to_date(jpn_gdp)
jpn_gdp_date

,Time,GDP
30,2017-01-01,0.740644
31,2017-04-01,0.260081
32,2017-07-01,0.797642
33,2017-10-01,0.402805
34,2018-01-01,-0.034943
35,2018-04-01,0.051794
36,2018-07-01,-0.653422
37,2018-10-01,0.453693
38,2019-01-01,0.577410
39,2019-04-01,0.077577


In [16]:
import datetime
from datetime import datetime

def filled_dataframe(currency):
    time_array = pd.date_range(start="1/1/2018", freq="1H", periods=26304)
    gdp_array = np.zeros(26304)
    gdp_index = 0
    index = 0
    previous = 0
    for i in currency.iterrows():
        currency_datetime = datetime.strptime(i[1]["Time"], "%Y-%m-%d")
        while index < 26304 and gdp_index < 26304 and time_array[index] < currency_datetime:
            gdp_array[gdp_index] = previous
            gdp_index += 1
            index += 1
        previous = i[1]["GDP"]
    while gdp_index < 26304:
        gdp_array[gdp_index] = currency["GDP"].iloc[-1]
        gdp_index += 1
    new_df = pd.DataFrame({"Time": time_array, "GDP": gdp_array})
    return new_df

print(filled_dataframe(jpn_gdp_date))

                     Time       GDP
0     2018-01-01 00:00:00 -0.034943
1     2018-01-01 01:00:00 -0.034943
2     2018-01-01 02:00:00 -0.034943
3     2018-01-01 03:00:00 -0.034943
4     2018-01-01 04:00:00 -0.034943
...                   ...       ...
26299 2020-12-31 19:00:00  5.295471
26300 2020-12-31 20:00:00  5.295471
26301 2020-12-31 21:00:00  5.295471
26302 2020-12-31 22:00:00  5.295471
26303 2020-12-31 23:00:00  5.295471

[26304 rows x 2 columns]


In [17]:
jpn_gdp_datetime = filled_dataframe(jpn_gdp_date)
usd_gdp_datetime = filled_dataframe(usd_gdp_date)
cad_gdp_datetime = filled_dataframe(cad_gdp_date)
aud_gdp_datetime = filled_dataframe(aud_gdp_date)
gbp_gdp_datetime = filled_dataframe(gbp_gdp_date)
eur_gdp_datetime = filled_dataframe(eur_gdp_date)
nzd_gdp_datetime = filled_dataframe(nzd_gdp_date)
chf_gdp_datetime = filled_dataframe(chf_gdp_date)
jpn_gdp_datetime

,Time,GDP
0,2018-01-01 00:00:00,-0.034943
1,2018-01-01 01:00:00,-0.034943
2,2018-01-01 02:00:00,-0.034943
3,2018-01-01 03:00:00,-0.034943
4,2018-01-01 04:00:00,-0.034943
...,...,...
26299,2020-12-31 19:00:00,5.295471
26300,2020-12-31 20:00:00,5.295471
26301,2020-12-31 21:00:00,5.295471
26302,2020-12-31 22:00:00,5.295471
